In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json

import polygon 
import gpx

In [2]:
cp_filepath = "../data/geojson/the_bridle.json"
run_filepath = "../data/stravaGPX/AnotherBridleContribution.gpx"

In [3]:
def get_geojson(filepath):
    file = open(filepath, "r") 
    return json.load(file)

def get_coordinates_json(filepath):
    data = get_geojson(filepath)
    return data

def get_coordinates_list(filepath, p=True):
    data = get_coordinates_json(filepath)['features'][0]['geometry']['coordinates']
    if p:
        return data[0]
    return data

In [4]:
data = get_coordinates_list(cp_filepath)

In [5]:
f = gpx.coordinates_dataframe_from_gpx(run_filepath)

In [6]:
f.assign(isInside=lambda d: d.apply(polygon.row_inside_polygon, args=[data], axis=1))

,latitude,longitude,time,isInside
0,40.774487,-73.987554,2019-12-25 12:44:26,False
1,40.774487,-73.987554,2019-12-25 12:44:27,False
2,40.774487,-73.987554,2019-12-25 12:44:28,False
3,40.774487,-73.987554,2019-12-25 12:44:36,False
4,40.774511,-73.987555,2019-12-25 12:44:37,False
5,40.774538,-73.987548,2019-12-25 12:44:38,False
6,40.774598,-73.987531,2019-12-25 12:44:40,False
7,40.774623,-73.987525,2019-12-25 12:44:41,False
8,40.774737,-73.987486,2019-12-25 12:44:46,False
9,40.774779,-73.987482,2019-12-25 12:44:47,False
